## Colin's Pipeline Experiments
### Objectives:
- Create a SingleImagePipeline so we can run fast-iteration experiments on just a single image
- Create a CentraPipeline that calls SingleImagePipeline
- Modularize the soccer_net_pipeline and migrate all operations to SingleImagePipeline
- Run all systematic ops from CentralPipeline (more than one tracklet)

### Main benefit from creating a `BatchImagePipeline` and a `CentralPipeline`
Decoupling large-scale systematic training versus testing. The main pipeline trains the model, creates output processed images and does so by systematically traversing all the tracklets. I want to be able to see what pre-processing is happening to the image so I can understand what is being fed to the model. I also just want to be able to pass a single raw image to the model so I can get fast results. Right now everything is coupled together by all the tracklets, and I don't want to traverse all of them or even a single tracklet, but maybe only a single image from a single tracklet.

### Quick Run Block

## Code

In [1]:
import sys
from pathlib import Path
import os

sys.path.append(str(Path.cwd().parent.parent))
print(str(Path.cwd().parent.parent))
print("Current working directory: ", os.getcwd())

from ModelDevelopment.CentralPipeline import CentralPipeline
from ModelDevelopment.ImageBatchPipeline import ImageBatchPipeline
from DataProcessing.DataPreProcessing import DataPaths

c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition
Current working directory:  c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\ModelDevelopment\experiments


In [2]:
# TODO: Add tracklets_override to pass a list of tracklets to process; tracklets_override=["8", "9", "10"]
# NOTE: It is recommended you delete the entire processed_data/{challenge/test/train} before running this to 100% avoid issues with old data.
# Furthermore, you should always restart your kernel before every new run because sometimes there are problems with paths
pipeline = CentralPipeline(
  num_tracklets=12,
  #num_images_per_tracklet=1,
  input_data_path=DataPaths.TRAIN_DATA_DIR.value,
  output_processed_data_path=DataPaths.PROCESSED_DATA_OUTPUT_DIR_TRAIN.value,
  common_processed_data_dir=DataPaths.COMMON_PROCESSED_OUTPUT_DATA_TRAIN.value,
  gt_data_path=DataPaths.TRAIN_DATA_GT.value,
  single_image_pipeline=False,
  display_transformed_image_sample=False, # NOTE: DO NOT USE. Code is parallelized so we cannot show images anymore. Code breaks, but first one will show if True.
  num_image_samples=1,
  use_cache=False, # Better set to false, not stable
  suppress_logging=False,
  num_workers=6
  )

2025-03-21 23:10:38 [INFO] DataPreProcessing initialized. Universe of available data paths:
2025-03-21 23:10:38 [INFO] ROOT_DATA_DIR: c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted
2025-03-21 23:10:38 [INFO] TEST_DATA_GT: c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\test\test_gt.json
2025-03-21 23:10:38 [INFO] TRAIN_DATA_GT: c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\train\train_gt.json
2025-03-21 23:10:38 [INFO] TEST_DATA_DIR: c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\test\images
2025-03-21 23:10:38 [INFO] TRAIN_DATA_DIR: c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\train\images
2025-03-21 23:10:38 [INFO] CHALLENGE_DATA_DIR: c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extr

In [3]:
pipeline.run_soccernet(
  run_soccer_ball_filter=True,
  generate_features=True,
  run_filter=True,
  run_legible=True,
  run_legible_eval=True,
  run_pose=True,
  run_crops=True,
  run_str=True,
  run_combine=True,
  run_eval=True)

2025-03-21 23:10:38 [INFO] Running the SoccerNet pipeline.
2025-03-21 23:10:38 [INFO] Using double parallelization: multiprocessing + CUDA batch processing.


Processing tracklets (CUDA + CPU):   0%|          | 0/12 [00:00<?, ?it/s]

2025-03-21 23:10:54 [INFO] DEBUG Number of images per tracklet (should be < max (1400+)): 578
2025-03-21 23:10:54 [INFO] Creating placeholder data files for Soccer Ball Filter.
2025-03-21 23:10:54 [INFO] Creating placeholder data files for Legibility Classifier.
2025-03-21 23:10:54 [INFO] Removed cached tracklet feature file (use_cache: False): c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\processed_data\train\0\features.npy
2025-03-21 23:10:54 [INFO] Removed cached tracklet feature file (use_cache: False): c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\processed_data\train\1\features.npy
2025-03-21 23:10:54 [INFO] Removed cached tracklet feature file (use_cache: False): c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\processed_data\train\2\features.npy
2025-03-21 23:10:54 [INFO] Removed cached tracklet feature file (use_cache: False): c:\Users\colin\OneDrive\De

Phase 1: Data Pre-Processing Pipeline Progress:   0%|          | 0/12 [00:00<?, ?it/s]

2025-03-21 23:10:54 [INFO] Found 0 balls, Ball list: []
2025-03-21 23:10:54 [INFO] Found 0 balls, Ball list: []
2025-03-21 23:10:54 [INFO] Found 0 balls, Ball list: []
2025-03-21 23:10:54 [INFO] Found 0 balls, Ball list: []
2025-03-21 23:10:54 [INFO] Found 0 balls, Ball list: []
2025-03-21 23:10:54 [INFO] Found 0 balls, Ball list: []


...

Generating crops:  47%|████▋     | 409/864 [00:00<00:00, 4076.12it/s]


skipping c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\train\images\2\2_74.jpg, unreliable points
skipping c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\train\images\2\2_75.jpg, unreliable points
skipping c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\train\images\2\2_76.jpg, unreliable points
skipping c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\train\images\3\3_105.jpg, unreliable points
skipping c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\train\images\3\3_153.jpg, unreliable points
skipping c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\train\images\3\3_37.jpg, unreliable pointsskipping c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\jersey

Generating crops: 100%|██████████| 864/864 [00:00<00:00, 3518.77it/s]

skipping c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\train\images\7\7_541.jpg, unreliable points
skipping c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\train\images\7\7_562.jpg, unreliable points
skipping c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\train\images\7\7_563.jpg, unreliable points
skipping c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\train\images\7\7_564.jpg, unreliable points
skipping c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\train\images\7\7_579.jpg, unreliable points
skipping c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\train\images\7\7_584.jpg, unreliable points
skipping c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\je

2025-03-21 23:17:32 [INFO] Additional keyword arguments: {'charset_test': '0123456789'}


2025-03-21 23:17:32 [ERROR] C:\Users\colin\miniconda3\envs\parseq2\lib\site-packages\timm\models\helpers.py:7: FutureWarning: Importing from timm.models.helpers is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
C:\Users\colin\miniconda3\envs\parseq2\lib\site-packages\lightning_fabric\utilities\cloud_io.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling.